In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group, flag
from spytial import relationalizer, RelationalizerBase, Atom, Relation

import dd
print(dd.__version__)

0.6.0


## DD is a standard Python lib for BDDs

- But no clear obv visualizer beyond dumping to network X and dot graphs.
- https://github.com/tulip-control/dd?tab=readme-ov-file



In [2]:
from dd.autoref import BDD

bdd = BDD()
bdd.declare('x', 'y', 'z')
u = bdd.add_expr(r'(x /\ y) \/ ~z')

# Map variable indices to names for readability
var_names = {idx: name for name, idx in bdd.vars.items()}

print("BDD node structure (node_id: var_name, low ->, high ->):")
for node, (var, low, high) in bdd._bdd._succ.items():
    var_label = var_names.get(var, str(var))
    print(f"Node {node}: var={var_label}, low={low}, high={high}")

print(f"\nBDD terminal info:")
print(f"False terminal: {bdd.false.node}")
print(f"True terminal: {bdd.true.node}")
print(f"Root node u: {u.node}")

print(f"\nAll referenced node IDs in relations:")
all_refs = set()
for node, (var, low, high) in bdd._bdd._succ.items():
    if low is not None:
        all_refs.add(low)
    if high is not None:
        all_refs.add(high)
print(f"Referenced IDs: {sorted(all_refs)}")

print(f"\nNodes that exist in _succ: {sorted(bdd._bdd._succ.keys())}")

#bdd.dump('bdd.dot')

# # Dump to JSON
# filename = 'bdd.json'
# roots = dict(u=u)
# bdd.dump(filename, roots)

BDD node structure (node_id: var_name, low ->, high ->):
Node 1: var=3, low=None, high=None
Node 2: var=x, low=-1, high=1
Node 3: var=y, low=-1, high=1
Node 4: var=x, low=-1, high=3
Node 5: var=z, low=-1, high=1
Node 6: var=y, low=-5, high=1
Node 7: var=x, low=-5, high=6

BDD terminal info:
False terminal: -1
True terminal: 1
Root node u: 7

All referenced node IDs in relations:
Referenced IDs: [-5, -1, 1, 3, 6]

Nodes that exist in _succ: [1, 2, 3, 4, 5, 6, 7]


In [4]:
@relationalizer(priority=101)
class BDDRelationalizer(RelationalizerBase):
    def can_handle(self, obj):
        return isinstance(obj, BDD)

    def relationalize(self, obj, walker_func):
        atoms = []
        relations = []
        
        # Build variable index to name mapping from BDD object
        var_names = {idx: name for name, idx in obj.vars.items()}
        seen = set()
        
        # Get terminal node IDs from dd library constants
        false_id = obj.false.node  # Usually -1
        true_id = obj.true.node    # Usually 1
        
        # Process all nodes from BDD structure
        for node_id, (var_idx, low_child, high_child) in obj._bdd._succ.items():
            # Check if this is a terminal node (no children)
            if low_child is None and high_child is None:
                # This is a terminal node
                if node_id == true_id:
                    atoms.append(Atom(id=str(node_id), type='Terminal', label='True'))
                elif node_id == false_id:
                    atoms.append(Atom(id=str(node_id), type='Terminal', label='False'))
                else:
                    # Unknown terminal - shouldn't happen in dd but handle gracefully
                    atoms.append(Atom(id=str(node_id), type='Terminal', label=f'Terminal_{node_id}'))
                seen.add(node_id)
            else:
                # This is a decision node
                if var_idx in var_names:
                    var_name = var_names[var_idx]
                else:
                    var_name = f"Unknown_var_{var_idx}"
                    
                atoms.append(Atom(id=str(node_id), type='BDDNode', label=var_name))
                seen.add(node_id)
                
                # Add relations for children
                if low_child is not None:
                    relations.append(Relation(name='low', atoms=[str(node_id), str(low_child)]))
                if high_child is not None:
                    relations.append(Relation(name='high', atoms=[str(node_id), str(high_child)]))
        
        # Add atoms for any missing referenced targets
        for relation in relations:
            for target_id in relation.atoms[1:]:  # Skip source node
                # Handle both string and integer target IDs
                if target_id not in seen:
                    # Handle serialized string constants from JSON dumps
                    if target_id == "T":
                        atoms.append(Atom(id="T", type='Terminal', label='True'))
                        seen.add("T")
                    elif target_id == "F":
                        atoms.append(Atom(id="F", type='Terminal', label='False'))
                        seen.add("F")
                    else:
                        try:
                            target_int = int(target_id)
                        except ValueError:
                            # Non-numeric, non-T/F string - handle gracefully
                            atoms.append(Atom(id=str(target_id), type='Unknown', label=f'Unknown_{target_id}'))
                            seen.add(target_id)
                            continue
                        
                        # Handle integer targets
                        if target_int == true_id:
                            atoms.append(Atom(id=str(target_int), type='Terminal', label='True'))
                        elif target_int == false_id:
                            atoms.append(Atom(id=str(target_int), type='Terminal', label='False'))
                        elif target_int < 0:
                            # Complemented node (negative ID)
                            pos_id = abs(target_int)
                            
                            # Try to get variable name from positive node
                            if pos_id in obj._bdd._succ:
                                pos_var_idx = obj._bdd._succ[pos_id][0]
                                if pos_var_idx in var_names:
                                    pos_var_name = var_names[pos_var_idx]
                                    label = f'NOT({pos_var_name})'
                                else:
                                    label = f'NOT(var_{pos_var_idx})'
                            elif pos_id == abs(false_id):
                                label = 'NOT(False)'
                            elif pos_id == abs(true_id):
                                label = 'NOT(True)'
                            else:
                                label = f'NOT({pos_id})'
                            
                            atoms.append(Atom(id=str(target_int), type='Complemented', label=label))
                        else:
                            # Positive integer that's not a known terminal
                            atoms.append(Atom(id=str(target_int), type='External', label=f'External_{target_int}'))
                        
                        seen.add(target_int)
        
        return atoms, relations

# Test the relationalizer
print("Testing BDD relationalizer:")
atoms, relations = BDDRelationalizer().relationalize(bdd, None)
print(f"Generated {len(atoms)} atoms and {len(relations)} relations")

# Show atoms with their labels
print("\nAtoms:")
for atom in sorted(atoms, key=lambda x: int(x.id) if x.id.lstrip('-').isdigit() else 999):
    print(f"  {atom.id}: {atom.label} ({atom.type})")

# Show relations
print(f"\nRelations:")
for rel in relations:
    print(f"  {rel.name}: {rel.atoms[0]} -> {rel.atoms[1]}")

# Validation scenario
result = diagram(bdd)
# print(f"\nGenerated BDD diagram: {result}")

# # Verify HTML file was created and contains valid content
# import os
# assert os.path.exists(result)
# with open(result, 'r') as f:
#     content = f.read()
#     assert 'html' in content.lower() and len(content) > 1000
# print("✓ BDD visualization works")

Testing BDD relationalizer:
Generated 19 atoms and 12 relations

Atoms:
  -5: NOT(z) (Complemented)
  -5: NOT(z) (Complemented)
  -1: False (Terminal)
  -1: False (Terminal)
  -1: False (Terminal)
  -1: False (Terminal)
  1: True (Terminal)
  1: True (Terminal)
  1: True (Terminal)
  1: True (Terminal)
  1: True (Terminal)
  2: x (BDDNode)
  3: y (BDDNode)
  3: External_3 (External)
  4: x (BDDNode)
  5: z (BDDNode)
  6: y (BDDNode)
  6: External_6 (External)
  7: x (BDDNode)

Relations:
  low: 2 -> -1
  high: 2 -> 1
  low: 3 -> -1
  high: 3 -> 1
  low: 4 -> -1
  high: 4 -> 3
  low: 5 -> -1
  high: 5 -> 1
  low: 6 -> -5
  high: 6 -> 1
  low: 7 -> -5
  high: 7 -> 6
